In [ ]:
import pandas as pd
import missingno as msno

## Vérification de la situation des données manquantes

In [ ]:
file_path = '/Users/ningyu/Desktop/Projets_data/ETL/0058127-231120084113126.csv'
df = pd.read_csv(file_path, delimiter='\t')

In [ ]:
msno.matrix(df)

## Traitement des données manquantes, conservation des colonnes valides

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName('describe_data')\
        .getOrCreate()

df = spark.read\
          .format('csv')\
          .option("header", "true")\
          .option("mode", "FAILFAST")\
          .option("inferSchema", "true")\
          .option("sep", "\t")\
          .load(file_path)

df.printSchema()

Selon le dictionnaire des données et la documentation sur la qualité des données, j'ai choisi les colonnes suivantes

In [ ]:
columns_to_keep = ['scientificName',
                       'countryCode',
                       'decimalLatitude',
                       'decimalLongitude',
                       'eventDate',
                       'occurrenceStatus',
                       'basisOfRecord',
                       'datasetKey',
                       'kingdom',
                       'phylum',
                       'class',
                       'order',
                       'family',
                       'genus',
                       'species',
                       'taxonRank',
                       'occurrenceID',
                       'individualCount']
df_clean_columns = df.select(columns_to_keep)
df_cleaned = df_clean_columns.na.drop(subset=['occurrenceID', 'eventDate', 'decimalLatitude', 'decimalLongitude'])

In [ ]:
df_pd = df_cleaned.toPandas()
msno.matrix(df_pd)

Pour les valeurs manquantes de l'IndividualCount, j'ai utilisé 1 comme substitut, car il y a des informations de coordonnées.

In [ ]:
summary = df_pd.describe(include="all").T
summary

À partir des données statistiques, nous pouvons constater que :
- scientificName n'est pas une valeur unique, il est nécessaire de filtrer uniquement les données pour "Panthera leo (Linnaeus, 1758)".
- occurrenceID comporte des doublons et doit être dédupliqué.
- Les données de coordonnées ne dépassent pas les valeurs extrêmes, elles n'ont donc pas besoin d'être traitées.

In [ ]:
value_counts = df_pd['scientificName'].value_counts()
value_counts

D'après les données précédentes, "Panthera leo (Linnaeus, 1758)" ne représente qu'une partie des données.

In [ ]:
lion_data = df_pd[df_pd['scientificName'] == 'Panthera leo (Linnaeus, 1758)']

In [ ]:
lion_data_unique = lion_data.drop_duplicates(subset='occurrenceID')

In [ ]:
summary = lion_data_unique.describe(include="all").T
summary